# Cross validating Keras models

In [ ]:
import os

import matplotlib.pyplot as plt

from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.models import Sequential
from keras.utils import np_utils
from sklearn.datasets import load_digits
from sklearn.model_selection import ShuffleSplit

from faculty_xval.validation import JobsCrossValidator
from faculty_xval.utilities import job_name_to_job_id

In [ ]:
REFERENCE_DIR = "foo"
JOB_NAME = "bar"

**Note:** In the code cell above, the definitions of `REFERENCE_DIR` and `JOB_NAME` change depending on how `faculty-xval` was installed:

* If you followed the **User installation instructions**, choose any `REFERENCE_DIR` for saving the results of cross validation. Running this notebook will create a sub-directory labelled with the current date and time, so that the output data are never overwritten. Make sure to specify `JOB_NAME` as `"cross_validation"`.

* If you followed the **Developer installation instructions**, create a directory for saving the results of cross validation in your personal directory (`/project/{USER_NAME}/temp`). Then select `REFERENCE_DIR` and `JOB_NAME` as follows:
```python
REFERENCE_DIR = "/project/{}/temp/".format(
    os.environ["USER_NAME"]
)
JOB_NAME = "cross_validation_{}".format(
    os.environ["USER_NAME"]
)
```

In [ ]:
# A quick check...
if not os.path.isdir(REFERENCE_DIR):
    raise OSError((
        "Path {} cannot be found "
        + "or is not a directory"
    ).format(REFERENCE_DIR))

In [ ]:
N_SPLITS = 10
TEST_SIZE = 0.25

N_NODES = 40
ACTIVATIONS = ["relu", "softmax"]
LOSS = "categorical_crossentropy"
OPTIMIZER = "adam"
FIT_KWARGS = {
    "epochs": 16,
    "batch_size": 32,
    "verbose": 0
}

NUM_SUBRUNS = 3

## Initialise the cross validator

In [ ]:
job_id = job_name_to_job_id(JOB_NAME)
cross_validator = JobsCrossValidator(job_id, REFERENCE_DIR)

## Load the data

In [ ]:
dataset = load_digits()
features = dataset["data"]
targets = dataset["target"]

In [ ]:
# Convert targets to one-hot encoding.
targets = np_utils.to_categorical(targets)

In [ ]:
print("Features:")
plt.imshow(features[3].reshape(8,8))
plt.show()

print("Targets:")
print(targets[3])

In [ ]:
split_generator = ShuffleSplit(
    n_splits=N_SPLITS,
    test_size=TEST_SIZE
).split(features)

## Define the Keras model

In [ ]:
model = Sequential()
model.add(Dense(
    N_NODES,
    input_shape=(features.shape[1],)
))
model.add(Activation(ACTIVATIONS[0]))
model.add(Dense(targets.shape[1]))
model.add(Activation(ACTIVATIONS[1]))

In [ ]:
model.compile(loss=LOSS, optimizer=OPTIMIZER)

## Perform cross validation

In [ ]:
cross_validator.run(
    model,
    [features],
    [targets],
    split_generator,
    NUM_SUBRUNS,
    fit_kwargs=FIT_KWARGS
)